<a href="https://colab.research.google.com/github/mdeniz20/NLP-0/blob/main/ai_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Chain

In [ ]:
!pip install -U langchain-cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnableSequence
import getpass
import os
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [ ]:
class TestLangchain:
    def add_one(self, x):
        return x + 1
    def multiply_by_two(self, x):
        return x * 2


    def run(self, x):
      runnable1 = RunnableLambda(self.add_one)
      runnable2 = RunnableLambda(self.multiply_by_two)
      sequence = runnable1 | runnable2
      return sequence.invoke(x)

    def run_alt(self, x):
      runnable1 = RunnableLambda(lambda x: x + 1)
      runnable2 = RunnableLambda(lambda x: x * 2)
      sequence = runnable1 | runnable2
      return sequence.invoke(x)

    def run_alt2(self, x):
      runnable1 = RunnableLambda(lambda x: x + 1)
      runnable2 = RunnableLambda(lambda x: self.multiply_by_two(x))
      runnable3 = RunnableLambda(lambda x: f"The result is {x}")
      sequence = RunnableSequence(first =  runnable1, middle = [runnable2], last=runnable3)
      return sequence.invoke(x)

test_chain = TestLangchain()
print(test_chain.run(2))
print(test_chain.run_alt(2))
print(test_chain.run_alt2(2))


6
6
The result is 6


In [ ]:
class Conversation:
  __messages = []
  __chat = None
  def __init__(self, *, api_key: str = "", model: str = "command-light", system_message: str = ""):
    if system_message:
      self.__add_message(SystemMessage(content=system_message))
    if api_key:
      os.environ["COHERE_API_KEY"] = api_key
    else:
      os.environ["COHERE_API_KEY"] = getpass.getpass()

    self.__chat = ChatCohere(model=model)

  def set_model(self, model: str) -> None:
    self.__chat = ChatCohere(model=model)
  def get_model(self) -> str:
    return self.__chat.model

  def __add_message(self, message) -> None:
    self.__messages.append(message)

  def add_message(self, message) -> None:
    self.__add_message(message)
  def get_messages(self):
    return self.__messages

  def invoke_chat(self, message) -> str:
    self.__add_message(HumanMessage(content=message))
    response = self.__chat.invoke(self.__messages)
    self.__add_message(AIMessage(content=response.content))
    return response

  def chat(self, message) -> str:
    return self.invoke_chat(message).content
ai = Conversation()

··········


In [ ]:
ai.set_model("command-r-plus")
print(ai.chat("how do you find those jokes?"))

As an AI language model, I have been trained on vast amounts of text data, including jokes, stories, books, and conversations. When you ask me to tell a joke, I search my database for relevant humor and select the most appropriate response based on the context and your request. 

It's an automated process where I analyze the input, process the request, and generate a response that aligns with your query. In this case, I searched for jokes related to lawyers and computer engineers and presented the ones that matched the criteria. 

So, when you ask me to "tell a joke," I'm happy to oblige and provide some laughter and entertainment!


In [ ]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere.chat_models import ChatCohere

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.pydantic_v1 import BaseModel, Field

os.environ["TAVILY_API_KEY"] = "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"

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."

class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")
internet_search.args_schema = TavilySearchInput

# LLM
llm = ChatCohere(model="command-r-plus", temperature=0)

# Preamble
preamble = """
You are an expert who answers the user's question with the most relevant datasource. You are equipped with an internet search tool and a special vectorstore of information about how to write good essays.
"""

# Prompt template
prompt = ChatPromptTemplate.from_template("How much tl needed to buy one dollar?")

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=llm,
    tools=[internet_search],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)

response = agent_executor.invoke({
    "input": "I want to write an essay. Any tips?",
    "preamble": preamble,
})

print(response['output'])



> Entering new AgentExecutor chain...

I will search for the current exchange rate between the dollar and the Turkish lira.
{'tool_name': 'internet_search', 'parameters': {'query': 'exchange rate dollar to turkish lira'}}
[{'url': 'https://www.xe.com/currencyconverter/convert/?From=USD&To=TRY', 'content': '1 USD to TRY - Convert US Dollars to Turkish Lire\nXe Currency Converter\n1.00 US Dollar =\n28.904216 Turkish Lire\n1 TRY = 0.0345970 USD\nConvert US Dollar to Turkish Lira\nConvert Turkish Lira to US Dollar\nUSD to TRY Chart\n1 USD = 0 TRY\n1 US Dollar to Turkish Lira stats\n Popular US Dollar (USD) Currency Pairings\nUSD to USD\nUSD to EUR\nUSD to GBP\nUSD to JPY\nUSD to CAD\nUSD to AUD\nUSD to CHF\nUSD to CNY\n Over 70 million downloads worldwide\n4.5/5, 2.2k ratings\n3.8/5, 90.8k ratings\n4.7/5, 41.5k ratings\nLanguage\nTransfer Money\nApps\nTools & Resources\nXE Business\nCompany Info\nConsent Manager The currency symbol is $.\nTRY - Turkish Lira\nOur currency rankings show th

# Logger Class

In [1]:
import os
import datetime
from datetime import timezone, timedelta

class Logger:
  """Helps you to trace what is happening in your code"""
  dir_name: str
  file_path: str
  def __init__(self, dir_name):
    os.system(f"mkdir {dir_name}")
    self.dir_name = dir_name

    current_time = self.get_current_time()
    filename = current_time + ".txt"
    self.file_path = dir_name+"/"+filename
    os.system(f"touch {self.file_path}")

  def log(self, input: str) -> None:
    with open(self.file_path, 'a') as file:
      current_time = self.get_current_time("%H:%M::%S")
      file.write(current_time + " -> " + input + "\n")

  def get_current_time(self, format = "%d-%m-%Y_%H:%M:%S"):
        # Get current date and time
    current_date_time = datetime.datetime.now()

    # Turkey is UTC+3
    timezone_offset = timedelta(hours=3)
    turkey_time = current_date_time + timezone_offset

    return turkey_time.strftime(format)

  def clear_all_in_dir(self):
    os.system(f"rm {self.dir_name}/*")

# Clear Logs

In [2]:
Logger("logs").clear_all_in_dir()

# Installations

In [3]:
!pip install langchainhub
!pip install -U langchain-cohere
!pip install wikipedia
%pip install --upgrade --quiet  langchain-community
!pip install mysql-connector-python
!apt-get -y install mysql-server
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY 'root';FLUSH PRIVILEGES;"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1

In [4]:
!pip install gradio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.8 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=ec22c8c5

# Environment Variables

In [5]:
from google.colab import userdata

os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["COHERE_API_KEY"] = userdata.get('COHERE_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
os.environ["WEATHER_API_KEY"] = userdata.get('weather_api')
os.environ["ALPHAVANTAGE_STOCK_API_KEY"] = userdata.get('ALPHAVANTAGE_STOCK_API_KEY')
os.environ["AV_EXCH_API_KEY"] = userdata.get('AV_EXCH_API_KEY')
os.environ["STABLE_API_KEY"] = userdata.get('STABLE_API_KEY')


# Imports

In [6]:
import os
import mysql.connector
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import Tool, StructuredTool
from langchain_cohere import ChatCohere
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain.schema import SystemMessage
from pydantic import BaseModel
import requests
import datetime
from datetime import timezone, timedelta
import base64
import uuid
from enum import Enum
import gradio as gr

# Image Generator

In [7]:
class Size:
  """Image size class."""
  height: int
  width: int
  def __init__(self, height, width):
    self.width = width
    self.height = height

class ImageSize(Enum):
  """Image size enumeration."""
  #heightxwidth
  STANDARD = Size(1024, 1024)
  WIDE = Size(640, 1536)
  LONG = Size(1536, 640)

class ImageGenerator:
  """Generates images using the Stable Diffusion API."""
  engine_id: str = None
  api_host: str = None
  api_key: str = None

  def __init__(self, api_host: str = None, api_key: str = None, engine_id:str = "stable-diffusion-v1-6"):
    self.engine_id = engine_id
    self.api_host = api_host if api_host else os.getenv("API_HOST", "https://api.stability.ai")
    self.api_key = api_key if api_key else os.getenv("STABLE_API_KEY")
    if self.api_key is None:
      raise Exception("Missing Stability API key.")

  def invoke(self,
      prompt: str,
      scale: int = 7,
      steps: int = 30,
      size: ImageSize = ImageSize.STANDARD,
      custom_size: Size = None,
      samples: int = 1, path = "./sd_generated_images"):
    """Generates images using the Stable Diffusion API."""
    image_size = size.value
    if custom_size:
      image_size = custom_size

    response = requests.post(
    f"{self.api_host}/v1/generation/{self.engine_id}/text-to-image",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        },
        json={
            "text_prompts": [
                {
                    "text": f"{prompt}"
                }
            ],
            "cfg_scale": scale,
            "height": image_size.height,
            "width": image_size.width,
            "samples": samples,
            "steps": steps,
        },
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    data = response.json()
    if not os.path.exists(path):
        os.makedirs(path)
    for i, image in enumerate(data["artifacts"]):
        photo_id = str(uuid.uuid1())
        with open(f"{path}/v1_txt2img_{i}_id[{photo_id}].png", "wb") as f:
            f.write(base64.b64decode(image["base64"]))
    return f"The image has been generated with prompt {prompt}", f"{path}/v1_txt2img_{i}_id[{photo_id}].png"


# BasilGPT

In [22]:
cur = None

prompt = hub.pull("hwchase17/structured-chat-agent")
logger = Logger("logs")
balance: float = 100

db_name = ""
db_password = ""
db_user_name = ""

last_image_set = None
# Define function schemas for structured tools
class NoArgs(BaseModel):
    pass

class BalanceArgs(BaseModel):
    amount: float

class SearchArgs(BaseModel):
    query: str

class DataArgs(BaseModel):
    query: str

class SetDatabaseArgs(BaseModel):
    database_name: str
    user_name: str
    password: str

class ForecastWeatherArgs(BaseModel):
    city: str
    forecast_days: int
    hour: int

class CurrentWeahterArgs(BaseModel):
  city: str

class CurrencyExchangeRateArgs(BaseModel):
  from_currency: str
  to_currency: str

class ImageArgs(BaseModel):
  prompt: str
  height: int
  width: int

def get_connection():
    global cur, db_name, db_password, db_user_name
    logger.log("get_connection")
    try:
        mydb = mysql.connector.connect(
            host="localhost",
            user='root',
            password='root',
            database='kumhrs'
        )
        cur = mydb.cursor()
        return cur
    except mysql.connector.Error as err:
        logger.log(f"MySQL Connection Error: {err}")
        return None

def get_current_time():
    """Returns the current time in HH:MM AM/PM format."""
    logger.log("get_current_time")
    # Get current date and time
    current_date_time = datetime.datetime.now()
    # Turkey is UTC+3
    timezone_offset = timedelta(hours=3)
    turkey_time = current_date_time + timezone_offset
    return turkey_time.strftime("%I:%M %p ")

def get_current_balance() -> str:
    """Returns the current balance of the user in terms of dollars ($)."""
    logger.log("get_current_balance")
    global balance
    return f"{balance}$ in the account"

def add_balance(amount: float) -> float:
    """Adds the given amount to the user's balance, then returns the recent balance in terms of dollars ($)."""
    logger.log(f"add_balance({amount})")
    global balance
    balance += float(amount)
    return balance

def search_wiki(query: str) -> str:
    """Returns the search result from wikipedia.com."""
    logger.log(f"search_wiki({query})")
    from wikipedia import summary
    print(query)
    try:
        return summary(query, sentences=2)
    except:
        return f"No results found for {query}"

def search_web(query: str) -> str:
    """Returns the search result from Google with GoogleSearchAPIWrapper().run(query) method."""
    logger.log(f"search_web({query})")
    search = GoogleSearchAPIWrapper()
    return search.run(query)

def chat(message: str) -> str:
    """Placeholder chat function."""
    return message

def run_database_query(query: str) -> list:
    """Returns the result of the database query output."""
    try:
        mydb = mysql.connector.connect(
        host="localhost",
        user='root',
        password='root',
        database='kumhrs'
        )
        cursor = mydb.cursor()
        result = cursor.execute(query)
        return result
    except mysql.connector.Error as err:
        logger.log(f"MySQL Error: {err}")
        return []


def get_database_information() -> dict:
    """Returns the database's table names and their attributes' name and data types. If an empty dictionary returned, means the connaction is failed"""
    global db_name, db_password, db_user_name, cur
    logger.log("get_database_information")
    mydb = mysql.connector.connect(
      host="localhost",
      user='root',
      password='root',
      database='kumhrs'
    )
    cursor = mydb.cursor()
    cur = get_connection()
    try:
      results = {}
      cursor.execute("SHOW TABLES")
      tables = cursor.fetchall()

      for table in tables:
          cursor.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table[0]}'")
          columns = cursor.fetchall()

          results[table[0]] = columns
      return results
    except:
      return {}


def set_database_information(database_name: str, user_name: str, password: str) -> str:
    """Sets the database's required information. Returns whether the database is set or required information is missing"""
    global db_name, db_password, db_user_name
    logger.log(f"set_database_information(db_name:{database_name}, user_name:{user_name}, password:{password})")
    db_name = database_name
    db_user_name = user_name
    db_password = password

    try:
      # Connect to the database
      mydb = mysql.connector.connect(
        host="localhost",
        user='root',
        password = 'root',
        database='kumhrs'
      )
    except:
      return("Database information is missing.")
    global cur
    cur = mydb.cursor()
    print(cur)
    return "Database is ready"

def get_forecast_weather_condition(city:str, forecast_days:int, hour:int):
  """Returns a dictionary of forecast weather conditions of given city name."""
  logger.log(f"get_forecast_weather_condition(city:{city}, forecast_days:{forecast_days}, hour:{hour})")
  key = os.environ["WEATHER_API_KEY"]
  parameters = {
    "key": key,
    "q": city,
    "days": forecast_days,
    "hour": hour
  }
  response = requests.get(url="https://api.weatherapi.com/v1/forecast.json", params = parameters)

  if response.status_code == 200:
    data = response.json()
  else:
    return "Connection Failed"
  return data

def get_current_weather_condition(city:str):
  """Returns a dictionary of current weather conditions of given city name."""
  logger.log(f"get_current_weather_condition(city:{city})")
  key = os.environ["WEATHER_API_KEY"]
  parameters = {
    "key": key,
    "q": city,
  }
  response = requests.get(url="https://api.weatherapi.com/v1/current.json", params = parameters)
  if response.status_code == 200:
    data = response.json()
  else:
    return "Connection Failed"
  return data

def get_currency_exchange_rate(from_currency:str, to_currency:str)->dict:
  """
  Retrieves the current exchange rate from Alpha Vantage API for a given currency pair.

  Args:
  - from_currency (str): The base currency code.
  - to_currency (str): The target currency code.

  Returns:
  - dict: A dictionary containing the exchange rate details, including 'from_currency',
          'to_currency', 'exchange_rate', and other relevant information.
  """
  logger.log(f"get_currency_exchange_rate(from_currency:{from_currency}, to_currency:{to_currency})")
  key = os.environ["AV_EXCH_API_KEY"],
  parameters = {
    "apikey": key,
    "from_currency": from_currency,
    "to_currency": to_currency,
    "function": "CURRENCY_EXCHANGE_RATE"
  }
  final_url = requests.Request('GET', "https://www.alphavantage.co/query", params=parameters).prepare().url
  print(final_url)
  response = requests.get(url="https://www.alphavantage.co/query", params = parameters)

  if response.status_code == 200:
    data = response.json()
    return data
  else:
    return {"error": "Could not get data"}

def generate_image(prompt: str, height: int, width: int) -> str:
  """
  Generates an image based on a given prompt using the Stable Diffusion API.

  Args:
  - prompt (str): The text prompt or description for generating the image.
  - height (int): The desired height of the generated image in pixels.
  - width (int): The desired width of the generated image in pixels.

  Returns:
  - success_message (str): A message indicating how many images generated and saved where.

  This function logs the generation process with the specified prompt, height, and width,
  creates a size object based on the provided dimensions, and invokes the Stable Diffusion API
  through ImageGenerator to generate the image. It then returns the success message.
  """
  logger.log(f"generate_image(prompt:{prompt}, height:{height}, width:{width})")
  size = Size(height, width)
  response, image_path = ImageGenerator().invoke(prompt, custom_size= size)
  global last_image_set
  last_image_set = image_path
  return response



tools = [
    StructuredTool(
        name="Current Time",
        func=get_current_time,
        description="Useful for when you need to know the current time",
        args_schema=NoArgs
    ),
    StructuredTool(
        name="Search Wiki",
        func=search_wiki,
        description="Useful for when you need to know information about a topic",
        args_schema=SearchArgs
    ),
    StructuredTool(
        name="Current Balance",
        func=get_current_balance,
        description="Useful for when you need to know the current balance in dollars",
        args_schema=NoArgs
    ),
    StructuredTool(
        name="Add Balance",
        func=add_balance,
        description="Adds an amount to the user's balance",
        args_schema=BalanceArgs
    ),
    StructuredTool(
        name="Google Search",
        func=search_web,
        description="Search Google for recent results.",
        args_schema=SearchArgs
    ),
    StructuredTool(
        name="Run Database Query",
        func=run_database_query,
        description="Useful for when you need to run queries on database.",
        args_schema=DataArgs
    ),
    StructuredTool(
        name="Set Database Information",
        func=set_database_information,
        description="Useful for when you need to set the database information.",
        args_schema=SetDatabaseArgs
    ),
    StructuredTool(
        name="Get Database Information",
        func=get_database_information,
        description="Should be used after 'Set Database Information' tool is used. Returns the name of the tables, and their attributes names",
        args_schema=NoArgs
    ),
    StructuredTool(
        name = "Get Forecast Weather Condition",
        func = get_forecast_weather_condition,
        description = "Useful for when you need to know the current and forecast weather condition",
        args_schema = ForecastWeatherArgs
    ),
    StructuredTool(
        name = "Get Current Weather Condition",
        func = get_current_weather_condition,
        description = "Useful for when you need to know the current weather condition",
        args_schema = CurrentWeahterArgs
    ),
    StructuredTool(
        name = "Get Currency Exchange Rate",
        func = get_currency_exchange_rate,
        description = "Useful for when you need to know the current exchange rate",
        args_schema = CurrencyExchangeRateArgs
    ),
    StructuredTool(
        name = "Generate Image",
        func = generate_image,
        description = "Useful for when you need to generate an image",
        args_schema = ImageArgs
    )
]
"""
StructuredTool(
    name="Chat",
    func=chat,
    description="Useful for when you need to chat",
    args_schema=SearchArgs  # Assuming chat function takes a string message
),
"""
llm = ChatCohere(model="command-r-plus")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

agent_executer = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=False,
    handle_parsing_errors=True
)
suffix = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
"""
suffix = ""
initial_message = """
**ALWAYS** introduce yourself at the beginning of the conversation.
Welcome to HllGPT, your AI assistant designed to provide helpful answers using available tools. If I'm unable to answer your question, I'll retry. You can also ask me to edit your balance.

Database Tools Overview:
You have access to several database tools:
1. **Setting Database Information**: Use this tool to set database information into local variables.
2. **Getting Database Information**: Retrieve database information from the local variables.
3. **Running Queries**: Execute queries in the database and receive the results.

**Guidelines for Database Interaction:**
- **Verification of Database Information**: Always verify if necessary database information is available. If not, prompt the user to provide it. Never generate information that is missing.
- **Initial Database Setup**: At the beginning, search the entire database to gather table names and their attribute names with get_database_information method.
- **Responding to Queries**: Answer the user's questions based on the results obtained from running the `run_database_query(query)` function.


**Guidelines of Weather Interaction:**

**Deciding the Day Parameter:**: If the user asks the rain for tomorrow, the day parameter should be 2
**Assumption of Hour Parameter:** If the user did not specify the hour to look for, always send 0 to the hour parameter.
**Specifications of Day Parameter:** The day parameter is an integer number representing the number of days in the forecast. For example, if the user asks about the weather tomorrow, the day parameter should be 2.
**Specifications of Hour Parameter:** The hour parameter is an integer number in 24-hour format. For instance, if the current time is 8 am, the hour parameter will be 8; if it's 8 pm, the hour parameter will be 20.

**Guidelines of Exchange Rate Interaction:**
**Assumption of Currency Parameters:** If the user does not specify the currency codes (from_currency and to_currency), the function defaults to USD as the base currency and TRY as the target currency.
**Currency Code Specification:** Currency codes should follow the ISO 4217 standard. For example, "USD" for US Dollar, "EUR" for Euro, "GBP" for British Pound Sterling, "TRY" for Turkish Liras and 'tl'.
**Functionality:** The function retrieves the current exchange rate from the Alpha Vantage API for a specified currency pair using the CURRENCY_EXCHANGE_RATE function.
**API Key Requirement:** Ensure your Alpha Vantage API key (AV_EXCH_API_KEY) is securely stored and accessible as an environment variable.
**Error Handling:** If the function fails to retrieve data due to network issues or API errors, it returns None and prints an error message with the HTTP status code.

**Guideline for Generating Image Interaction:**
**ALWAYS** generate one image at one time.
**ALWAYS** tell the content of the generated image. The content will be provided with return value of the Generate Image function.
**Return Value of Generate Image function**: The function returns only success message. Image is not waited from this function. Image is saved and displayed by other front end and back end code, it is not your job to view the image.
**Image Size Specification:**
These are the available image sizes (height x width):
- 1024 x 1024
- 1152 x 896
- 896 x 1152
- 1216 x 832
- 1344 x 768
- 768 x 1344
- 1536 x 640
- 640 x 1536
By default, the image size is assumed to be 1024 x 1024 if not specified.
For specific sizes in terms of height x width:
- STANDARD: 1024 x 1024 if the user did not specify any size.
- WIDE: 640 x 1536  if the user specified it as: it is for wall paper or background or wide banner or it is wide.
- LONG: 1536 x 640 if the user specified it as: it is for social media or phone or it is long.
If the user does not specify the number of images to be generated, only one image will be generated. Generating more than one image is not supported, so if requested, generate one image.
If the user asks for more than one image, say that this feature will be awailable with some updates in the feature.
"""
memory.chat_memory.add_message(SystemMessage(content=initial_message))
def chat(user_input):
    memory.chat_memory.add_message(HumanMessage(content=user_input+suffix))
    response = agent_executer.invoke({"input": user_input+suffix})
    memory.chat_memory.add_message(AIMessage(content=response["output"]))
    return response['output'], last_image_set

interface = gr.Interface(
    fn=chat,
    inputs = ["text"],
    outputs = ["text", "image"]
)

interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://010db4c872b9c35042.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [31]:
"""
The weather in Ankara, Turkey, is currently sunny with a temperature of 30.5ºC or 86.9ºF. The humidity is relatively low at 29%, and the wind speed is 15.1 kph.
It is currently 04:15 PM.
The current exchange rate for USD to TRY is 32.99, last refreshed at 1:15 PM UTC today.
I've added $30 to your balance as requested. Your new balance is $160.
"""

"""
{
  "prompt1": "Generate an image for my AI presentation.",
  "prompt2": "What time is it?",
  "prompt3": "What is the weather like in Ankara?",
  "prompt4": "Check my balance.",
  "prompt5": "What is the current TL/USD exchange rate?",
  "prompt6": "Add 30 dollars to my balance."
}
"""


llm = ChatCohere(model="command-r-plus")
#resp = llm.invoke("""
#You are a prompt engineer. Your job is get the prompt and divide them into individual prompts.
#**ALWAYS** give the all prompts in json format and do not add noncode text
#The format should be like this:
#{
#  "prompt1": "this is prompt 1",
#  "prompt2": "this is prompt 2"
#}
#can you divide this prompt into separate prompts? But do not answer them.
#
#What is the weather like in Ankara, what time is it, check my balance, what tl/usd is also add 30$ into my balance.""").content

#print(resp)

print("""

The results are:
The weather in Ankara, Turkey, is currently sunny with a temperature of 30.5ºC or 86.9ºF. The humidity is relatively low at 29%, and the wind speed is 15.1 kph.
It is currently 04:15 PM.
The current exchange rate for USD to TRY is 32.99, last refreshed at 1:15 PM UTC today.
I've added $30 to your balance as requested. Your new balance is $160.

""")

resp = llm.invoke("""
You are a prompt engineer. Your job is to summarize the given text without losing its meaning and detail.
**ALWAYS** give the final answer only
**SUMMARIZE** the following text:
The weather in Ankara, Turkey, is currently sunny with a temperature of 30.5ºC or 86.9ºF. The humidity is relatively low at 29%, and the wind speed is 15.1 kph.
It is currently 04:15 PM.
The current exchange rate for USD to TRY is 32.99, last refreshed at 1:15 PM UTC today.
I've added $30 to your balance as requested. Your new balance is $160.
""")
print("Summarization: ",resp.content)



The results are:
The weather in Ankara, Turkey, is currently sunny with a temperature of 30.5ºC or 86.9ºF. The humidity is relatively low at 29%, and the wind speed is 15.1 kph.
It is currently 04:15 PM.
The current exchange rate for USD to TRY is 32.99, last refreshed at 1:15 PM UTC today.
I've added $30 to your balance as requested. Your new balance is $160.


Summarization:  The weather in Ankara, Turkey, is sunny with a temperature of 86.9ºF, low humidity of 29%, and a wind speed of 15.1 kph. The USD to TRY exchange rate is 32.99 as of 1:15 PM UTC today. Your balance has been updated with an additional $30, resulting in a new total of $160.


I provided the weather in Ankara, the time, the USD-TRY exchange rate, and confirmed adding $30 to a balance.


# Database

In [9]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password = "root"
)

cur = mydb.cursor()
def populate():
  cur.execute("CREATE DATABASE IF NOT EXISTS kumhrs")
  cur.execute("USE kumhrs")
  file = open("./db/sql.txt", "r")
  sql = file.read()
  cur.execute(sql)

# populate()
cur.execute("USE kumhrs")
cur.execute("SELECT COUNT(*) FROM Person")
print(cur.fetchall())
"""
cur.execute(
SELECT COLUMN_NAME
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Hospital';
)
print(cur.fetchall())

"""
try:
  results = {}
  cur.execute("SHOW TABLES")
  tables = cur.fetchall()
  print(f"tables: {tables}")
  for table in tables:
      cur.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table[0]}'")
      columns = cur.fetchall()
      print("columns:", columns)
      results[table[0]] = columns
  print(results)
  print(f"results: {results}")
except:
  {}

hi
[(56,)]
tables: [('Appointment',), ('Department',), ('Doctor',), ('Hospital',), ('MedicineDetail',), ('Patient',), ('Person',), ('Prescription',), ('PrescriptionMedicine',)]
columns: [('adate', 'date'), ('aid', 'int'), ('atime', 'time'), ('doctor_id', 'int'), ('patient_id', 'int'), ('status', 'varchar')]
columns: [('department_id', 'int'), ('dname', 'varchar'), ('hospital_id', 'int')]
columns: [('department_id', 'int'), ('doctor_id', 'int'), ('rid', 'int')]
columns: [('haddress', 'varchar'), ('hname', 'varchar'), ('hospital_id', 'int')]
columns: [('dosage', 'varchar'), ('medicine_name', 'varchar'), ('mid', 'int'), ('note', 'varchar'), ('quantity', 'int'), ('tpd', 'int'), ('tpu', 'int')]
columns: [('emergency_contact', 'varchar'), ('patient_id', 'int'), ('rid', 'int')]
columns: [('address', 'varchar'), ('bdate', 'date'), ('email', 'varchar'), ('gender', 'varchar'), ('insnumber', 'varchar'), ('name', 'varchar'), ('password', 'varchar'), ('pnumber', 'varchar'), ('rid', 'int')]
columns:

# Weather Test

In [7]:
import requests
from google.colab import userdata

def get_weather_condition(city:str, forecast_days:str, hour:str):
  """Returns a dictionary of weather conditions of given city name."""
  parameters = {
    "key": userdata.get('weather_api'),
    "q": city,
    "days": forecast_days,
    "hour": hour
  }
  response = requests.get(url="https://api.weatherapi.com/v1/forecast.json", params = parameters)

  if response.status_code == 200:
    data = response.json()
    print(data)
  else:
    print(response.status_code)

get_weather_condition("istanbul", "3", "1)")

{'location': {'name': 'Istanbul', 'region': 'Istanbul', 'country': 'Turkey', 'lat': 41.02, 'lon': 28.96, 'tz_id': 'Europe/Istanbul', 'localtime_epoch': 1721883816, 'localtime': '2024-07-25 8:03'}, 'current': {'last_updated_epoch': 1721883600, 'last_updated': '2024-07-25 08:00', 'temp_c': 27.5, 'temp_f': 81.6, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 12.8, 'wind_kph': 20.5, 'wind_degree': 48, 'wind_dir': 'NE', 'pressure_mb': 1006.0, 'pressure_in': 29.69, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 54, 'cloud': 3, 'feelslike_c': 28.6, 'feelslike_f': 83.4, 'windchill_c': 27.5, 'windchill_f': 81.6, 'heatindex_c': 28.6, 'heatindex_f': 83.4, 'dewpoint_c': 17.5, 'dewpoint_f': 63.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 7.0, 'gust_mph': 15.0, 'gust_kph': 24.1}, 'forecast': {'forecastday': [{'date': '2024-07-25', 'date_epoch': 1721865600, 'day': {'maxtemp_c': 30.0, 'maxtemp_f': 86.0, 'mintemp_c': 24.4,

# Gradio Test

In [93]:
import gradio as gr

def greet(name):
    if name == "image1":
        return f"Hello, {name}!", "./sd_generated_images/v1_txt2img_0_id[c413a6bc-4a70-11ef-93ab-0242ac1c000c].png"  # Return text and image path
    else:
        return f"Hello, {name}!", None  # Return only text if not 'image1'

demo = gr.Interface(
    fn=greet,
    inputs="text",
    outputs=["text", "image"],  # Specify both text and image outputs
)

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://76d29ddacfdd08016b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Exchange Test

In [47]:
import requests

def get_exchange(from_currency, to_currency):
  """
  Retrieves the current exchange rate from Alpha Vantage API for a given currency pair.

  Args:
  - from_currency (str): The base currency code.
  - to_currency (str): The target currency code.

  Returns:
  - dict: A dictionary containing the exchange rate details, including 'from_currency',
          'to_currency', 'exchange_rate', and other relevant information.
  """
  key = os.environ["AV_EXCH_API_KEY"],
  parameters = {
    "apikey": key,
    "from_currency": from_currency,
    "to_currency": to_currency,
    "function": "CURRENCY_EXCHANGE_RATE"
  }
  final_url = requests.Request('GET', "https://www.alphavantage.co/query", params=parameters).prepare().url
  print(final_url)
  response = requests.get(url="https://www.alphavantage.co/query", params = parameters)

  if response.status_code == 200:
    data = response.json()
    print(data)
get_exchange("USD", "TRY")

https://www.alphavantage.co/query?apikey=84UQSWBN5ZIZU7RG&from_currency=USD&to_currency=TRY&function=CURRENCY_EXCHANGE_RATE
{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'USD', '2. From_Currency Name': 'United States Dollar', '3. To_Currency Code': 'TRY', '4. To_Currency Name': 'Turkish Lira', '5. Exchange Rate': '32.96160000', '6. Last Refreshed': '2024-07-25 06:39:02', '7. Time Zone': 'UTC', '8. Bid Price': '32.96076000', '9. Ask Price': '32.96226000'}}


# SDXL Test

In [90]:
class Size:
  """Image size class."""
  height: int
  width: int
  def __init__(self, height, width):
    self.width = width
    self.height = height

class ImageSize(Enum):
  """Image size enumeration."""
  #heightxwidth
  STANDARD = Size(1024, 1024)
  WIDE = Size(640, 1536)
  LONG = Size(1536, 640)

class ImageGenerator:
  """Generates images using the Stable Diffusion API."""
  engine_id: str = None
  api_host: str = None
  api_key: str = None

  def __init__(self, api_host: str = None, api_key: str = None, engine_id:str = "stable-diffusion-v1-6"):
    self.engine_id = engine_id
    self.api_host = api_host if api_host else os.getenv("API_HOST", "https://api.stability.ai")
    self.api_key = api_key if api_key else os.getenv("STABLE_API_KEY")
    if self.api_key is None:
      raise Exception("Missing Stability API key.")

  def invoke(self,
      prompt: str,
      scale: int = 7,
      steps: int = 30,
      size: ImageSize = ImageSize.STANDARD,
      custom_size: Size = None,
      samples: int = 1, path = "./sd_generated_images"):
    """Generates images using the Stable Diffusion API."""
    image_size = size.value
    if custom_size:
      image_size = custom_size

    response = requests.post(
    f"{self.api_host}/v1/generation/{self.engine_id}/text-to-image",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        },
        json={
            "text_prompts": [
                {
                    "text": f"{prompt}"
                }
            ],
            "cfg_scale": scale,
            "height": image_size.height,
            "width": image_size.width,
            "samples": samples,
            "steps": steps,
        },
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    data = response.json()
    if not os.path.exists(path):
        os.makedirs(path)
    for i, image in enumerate(data["artifacts"]):
        photo_id = str(uuid.uuid1())
        with open(f"{path}/v1_txt2img_{i}_id[{photo_id}].png", "wb") as f:
            f.write(base64.b64decode(image["base64"]))
    return f"{i+1} images created and saved to the path {path}"

ImageGenerator().invoke("An astronout sitting on throne of the universe.", size = ImageSize.LONG)



'1 images created and saved to the path ./sd_generated_images'